<a href="https://colab.research.google.com/github/Yousof-Mahmoud/Computer-Vision-2022/blob/main/hybrid_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
#import scipy.stats as st
#from scipy.signal import convolve2d

In [3]:
# freqyency filters
def square_pad(source: np.ndarray, size_x: int, size_y: int, pad_value: int) -> np.ndarray:
   
    src = np.copy(source)
    x, y = src.shape

    out_x = (size_x - x) // 2
    out_xx = size_x - out_x - x

    out_y = (size_y - y) // 2
    out_yy = size_y - out_y - y

    return np.pad(src, ((out_x, out_xx), (out_y, out_yy)), constant_values=pad_value)

In [2]:
def frequency_filter(source: np.ndarray, kernel: np.ndarray) -> np.ndarray:
   
    src = np.copy(source)

    # Covert Image to gray scale
    src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

    # Convert Image to Frequency Domain
    # and decentralize the output
    ft_src = np.fft.fft2(src)
    ft_src_shifted = np.fft.fftshift(ft_src)

    # apply Kernel
    out = np.fft.ifftshift(ft_src_shifted * kernel)
    out = np.fft.ifft2(out)

    return np.abs(out)

In [4]:
def high_pass_filter(source: np.ndarray, size: int) -> np.ndarray:
  
    # Create a kernel with ones in the middle for high frequencies
    kernel = square_pad(np.ones((size, size)), source.shape[0], source.shape[1], 0)

    # Apply Kernel
    out = frequency_filter(source, kernel)

    return out

In [5]:
def low_pass_filter(source: np.ndarray, size: int) -> np.ndarray:
    
    # Create Kernel with ones on the edges for low frequencies
    kernel = 1 - square_pad(np.ones((size, size)), source.shape[0], source.shape[1], 0)

    # Apply Kernel
    out = frequency_filter(source, kernel)

    return out

In [8]:
#hybrid images 
def hybrid_image(src_img1 ,src_img2):
  min_shape =(min(src_img1.shape[0], src_img2.shape[0]),
              min(src_img1.shape[1], src_img2.shape[1]))
  src_img1_resized = cv2.resize(src_img1, min_shape, interpolation=cv2.INTER_AREA)
  src_img2_resized = cv2.resize(src_img2, min_shape, interpolation=cv2.INTER_AREA)

   # Apply filters
  image1_dft = high_pass_filter(source=src_img1_resized, size=20)
  image2_dft = low_pass_filter(source=src_img2_resized, size=15)
  hybrid_image = image1_dft + image2_dft
  cv2.imwrite('hybrid_image.png', hybrid_image)

In [9]:
src_img1= cv2.imread('/content/h1.png')
src_img2= cv2.imread('/content/h2.png')
hybrid=hybrid_image(src_img1 ,src_img2)